In [ ]:
import pandas as pd
import re
import time

In [ ]:
import requests

In [ ]:
import boto3
from botocore.exceptions import ClientError

# load data

## m3u playlist

In [ ]:
# load playlist
df = pd.read_csv('./playlist.csv')

## epg data

In [ ]:
# add here urls to your epg services
epg_urls = ['https://epgshare01.online/epgshare01/epg_ripper_DE1.txt',
            'https://epgshare01.online/epgshare01/epg_ripper_US1.txt',
            'https://epgshare01.online/epgshare01/epg_ripper_UK1.txt']

In [ ]:
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36"
}

response_combined = []

for url in epg_urls:
    print(f'Fetching {url}')
    response = requests.get(url, headers=headers)
    response_combined.append(response.text)


In [ ]:
# join the two responses
response_combined = '\n'.join(response_combined)


In [ ]:
# create list
epg_data = response_combined.split('\n')

# remove empty lines
epg_data = [line for line in epg_data if line]

epg_data

In [ ]:
len(epg_data)

# find epg entries (using LLM and AWS Bedrock)

In [ ]:
# Create a Bedrock Runtime client in the AWS Region you want to use.
client = boto3.client("bedrock-runtime", region_name="us-east-1")

# Set the model ID, e.g., Amazon Nova Lite.
model_id = "amazon.nova-lite-v1:0"

In [ ]:
def llm_response(client, model_id, epg_data, channel, country):

    user_message = f'''
        You are given a tv channel list and a channel name. 
        You provide the closest match to the channel name from the list.

        Tv channel list: {epg_data}
        Channel name: {channel}
        Country: {country}

        Firstly explain your response, followed by your final answer. You should follow the format 
        Explanation: [Explanation], Answer: [Answer],

        where [Answer] can be one entry from the tv channel list or "no match". 
    '''

    conversation = [
        {
            "role": "user",
            "content": [{"text": user_message}],
        }
    ]

    try:
        # Send the message to the model, using a basic inference configuration.
        response = client.converse(
            modelId=model_id,
            messages=conversation,
            inferenceConfig={"maxTokens": 512, "temperature": 0, "topP": 0.5},
        )

        # Extract and print the response text.
        response_text = response["output"]["message"]["content"][0]["text"]
        #print(response_text)

        # extract the last line
        response = response_text.split('\n')[-1].strip()

        # extract entry after :
        response = response.split(':')[-1].strip()

        # remove any ' or " characters
        response = response.replace("'", "")
        response = response.replace('"', "")
        
        return response
    
    except (ClientError, Exception) as e:
        print(f"ERROR: Can't invoke '{model_id}'. Reason: {e}")

In [ ]:
epg_ids = []

for ix, row in df.iterrows():

    ch = row['Channel Name']
    country = row['Country']
    
    response = llm_response(client, model_id, epg_data, ch, country)
    print(ch, response)

    if (response != "no match"):
        epg_ids.append((ch,response))


In [ ]:
# add to tvg id column based on first tuple entry
df['TVG ID'] = df['Channel Name'].map(dict(epg_ids))

# fill entries where no match has found 
df['TVG ID'] = df['TVG ID'].fillna('')

In [ ]:
# check empty TVG ID
mask = df['TVG ID'] == ''
df.loc[mask]

# Save data

In [ ]:
# Save the DataFrame to an M3U formatted file
m3u_file_path = "playlist.m3u"

with open(m3u_file_path, "w") as f:
    f.write("#EXTM3U\n")
    for _, row in df.iterrows():
        f.write(f'#EXTINF:-1 tvg-chno="{row["TVG Channel Number"]}" tvg-id="{row["TVG ID"]}" tvg-name="{row["TVG Name"]}" tvg-logo="{row["TVG Logo"]}" group-title="{row["Group Title"]}",{row["Channel Name"]}\n')
        f.write(f'{row["Stream URL"]}\n')

print(f"M3U8 file saved as {m3u_file_path}")